<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-preprocessed-data-frame" data-toc-modified-id="The-preprocessed-data-frame-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The preprocessed data frame</a></span></li><li><span><a href="#Baseline-classifier" data-toc-modified-id="Baseline-classifier-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Baseline classifier</a></span><ul class="toc-item"><li><span><a href="#Preprocess-data" data-toc-modified-id="Preprocess-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preprocess data</a></span></li><li><span><a href="#Train-a-Logistic-Regression-model" data-toc-modified-id="Train-a-Logistic-Regression-model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Train a Logistic Regression model</a></span></li><li><span><a href="#Classification-report" data-toc-modified-id="Classification-report-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Classification report</a></span></li><li><span><a href="#Train-and-eval-a-model-for-cpvs" data-toc-modified-id="Train-and-eval-a-model-for-cpvs-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Train and eval a model for cpvs</a></span><ul class="toc-item"><li><span><a href="#33" data-toc-modified-id="33-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>33</a></span></li><li><span><a href="#71" data-toc-modified-id="71-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>71</a></span></li></ul></li></ul></li><li><span><a href="#91" data-toc-modified-id="91-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>91</a></span></li></ul></div>

# The preprocessed data frame

In [1]:
import pandas as pd
from IPython.display import display

df = pd.read_csv("../data/processed/redflags.tsv", encoding="utf-8", sep="\t")
display(df)

,id,title,shortDescription,cpvs,AwCritLacksIndicator,AwCritMethodMissingIndicator,AwCritPaymentDeadlineCondIndicator,ContrDescCartellingIndicator,CountOfInvOpsNoCondIndicator,DeadlineIsTightIndicator,...,RenewalOfContractIndicator,TechCapEURefCondIndicator,TechCapExpertsExpCondManyYearsIndicator,TechCapGeoCondIndicator,TechCapMissingMinCondIndicator,TechCapRefCondExceedEstimValIndicator,TechCapRefCondManyYearsIndicator,TechCapSingleContractRefCondIndicator,cpvs_filtered,cleaned text
0,100003-2013,Magyar Tudományos Akadémia köztestületi intézm...,Szállítási szerződés alapján a Magyar Tudomány...,9123000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91,"['adásvételi', 'szerződés', 'a', 'keop-5', '6...."
1,100020-2019,Laboratóriumi eszközök GINOP-2.3.2-15-216-0001...,Adásvételi szerződés 1 darab elektronmikroszkó...,38432200|38511000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,"['m³', 'mennyiségű', 'kommunális', 'és', 'épít..."
2,100077-2019,Mikrohullámú rendszerhez eszközök beszerzése H...,Pont-pont mikrohullámú adatátviteli rendszer k...,31711422|35125000|80531200,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31,"['megbízási', 'keretszerződés', 'keret', 'a', ..."
3,100095-2019,MH EK sztentek beszerzése Hivatkozási szám: EK...,Magyar Honvédség Egészségügyi Központ sztentek...,33141230,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33,"['építési', 'beruházás', 'fidic', 'piros', 'kö..."
4,100234-2018,Keretmegállapodás Győr Megyei Jogú Város önkor...,Keretmegállapodás Győr Megyei Jogú Város önkor...,45233141|45233142,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45,"['szerződés', 'meghatározás', 'adásvételi', 'k..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156,247666-2013,Takarítás,Vállalkozási szerződés.,90900000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90,"['adás-vételi', 'szerződés', 'db', 'röntgen', ..."
3157,247693-2012,Szolgáltatási szerződés.,Esztergom Város Önkormányzatának tulajdonában ...,79713000,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79,"['adásvételi', 'szerződés', 'a', 'keop', '5.6...."
3158,247926-2020,Családbarát szülészet fejlesztése Hivatkozási ...,Családbarát szülészet fejlesztése – eszközbesz...,33100000|33152000|33157000|33158500|33190000|3...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33,"['vállalkozási', 'szerződés', 'a', 'szegedi', ..."
3159,247939-2014,Vállalkozási szerződés „Nagyvízi mederkezelési...,Nagyvízi mederkezelési tervek készítése.,71320000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,71,"['adásvételi', 'szerződés', 'teleszkópos', 'ra..."


# Baseline classifier
## Preprocess data

In [2]:
from collections import Counter

rf_features = ["AwCritLacksIndicator", "AwCritMethodMissingIndicator", "AwCritPaymentDeadlineCondIndicator", "ContrDescCartellingIndicator",
               "CountOfInvOpsNoCondIndicator", "DeadlineIsTightIndicator", "DurationLongOrIndefiniteIndicator", "FinAbEquityCondIndicator",
               "FinAbMissingMinCondIndicator", "FinAbRevenueCondExceedEstimValIndicator", "FinAbRevenueCondManyYearsIndicator",
               "FwAgFewParticipantsIndicator", "FwAgHighEstimatedValueIndicator", "FwAgLongDurationIndicator", "FwAgOneParticipantIndicator",
               "HighEstimatedValueIndicator", "OfferGuaranteeIsHighIndicator", "OpeningDateDiffersFromDeadlineIndicator",
               "PersSitMissingCondIndicator", "ProcTypeAcceleratedIndicator", "ProcTypeNegotiatedNoJustificationIndicator",
               "RenewalOfContractIndicator", "TechCapEURefCondIndicator", "TechCapExpertsExpCondManyYearsIndicator", "TechCapGeoCondIndicator",
               "TechCapMissingMinCondIndicator", "TechCapRefCondExceedEstimValIndicator",	"TechCapRefCondManyYearsIndicator",
               "TechCapSingleContractRefCondIndicator"]

# get the RedFlags features
df_feature_vecs = df[rf_features]
flag_dist = []
problematic = []
for feat in rf_features:
    vals = list(df[feat])
    vals = [1 if e > 0.0 else 0.0 for e in vals]
    flag_dist.append(sum(vals))
    if sum(vals) < 10.0:
        problematic.append(feat)

# get rid of unimportant indicators
important_indicators = set(rf_features).difference(set(problematic))
df_subset = df[important_indicators]

# convert flags into labels -> Good or Bad
summas = []
for idx, row in df_subset.iterrows():
    summas.append(sum(list(row)))
y = ["Good" if e < 0.5 else "Bad" for e in summas]
df["Category"] = y


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split



# corpus
corpus = list(df["cleaned text"])
corpus = [e.strip() for e in corpus]

vectorizer = TfidfVectorizer(analyzer="char_wb", sublinear_tf=True, ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)# train - test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)



## Train a Logistic Regression model
We chose LR because it is a good baseline model, in addition it handles imbalanced classes. WARNING: this is just the baseline model. Later, we are going to optimize everything.

In [4]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, solver='liblinear', C=5, penalty='l2',max_iter=2000, class_weight="balanced")
clf.fit(X_train, y_train)

LogisticRegression(C=5, class_weight='balanced', max_iter=2000, random_state=42,
                   solver='liblinear')

## Classification report

In [5]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.25      0.30      0.27       407
        Good       0.74      0.68      0.71      1174

    accuracy                           0.58      1581
   macro avg       0.49      0.49      0.49      1581
weighted avg       0.61      0.58      0.59      1581



## Train and eval a model for cpvs
First, let's see the distribution of cpvs.

In [6]:
from collections import Counter

cpvs = list(df['cpvs_filtered'])
cpvs = Counter(cpvs)
cpvs = {k: v for k, v in sorted(cpvs.items(), key=lambda item: item[1], reverse=True)}
print(cpvs)

{33: 506, 71: 350, 91: 248, 45: 242, 79: 242, 34: 203, 90: 190, 93: 160, 50: 138, 72: 126, 66: 116, 15: 105, 38: 86, 55: 71, 42: 64, 30: 53, 44: 43, 48: 42, 39: 42, 32: 40, 77: 39, 24: 26, 31: 16, 37: 13}


We are going to investigate only the first 3 cases.
### 33

In [7]:
from sklearn.linear_model import LogisticRegression

df33 = df[df["cpvs_filtered"] == 33]

y = list(df33["Category"])

# corpus
corpus = list(df33["cleaned text"])
corpus = [e.strip() for e in corpus]

vectorizer = TfidfVectorizer(analyzer="char_wb", sublinear_tf=True, ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)# train - test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf = LogisticRegression(random_state=42, solver='liblinear', C=5, penalty='l2',max_iter=2000, class_weight="balanced")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.28      0.28      0.28        81
        Good       0.66      0.66      0.66       172

    accuracy                           0.54       253
   macro avg       0.47      0.47      0.47       253
weighted avg       0.54      0.54      0.54       253



### 71

In [8]:
df71 = df[df["cpvs_filtered"] == 71]

y = list(df71["Category"])

# corpus
corpus = list(df71["cleaned text"])
corpus = [e.strip() for e in corpus]

vectorizer = TfidfVectorizer(analyzer="char_wb", sublinear_tf=True, ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)# train - test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf = LogisticRegression(random_state=42, solver='liblinear', C=5, penalty='l2',max_iter=2000, class_weight="balanced")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.32      0.14      0.19        50
        Good       0.72      0.88      0.79       125

    accuracy                           0.67       175
   macro avg       0.52      0.51      0.49       175
weighted avg       0.60      0.67      0.62       175



# 91

In [9]:
df91 = df[df["cpvs_filtered"] == 91]

y = list(df91["Category"])

# corpus
corpus = list(df91["cleaned text"])
corpus = [e.strip() for e in corpus]

vectorizer = TfidfVectorizer(analyzer="char_wb", sublinear_tf=True, ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)# train - test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf = LogisticRegression(random_state=42, solver='liblinear', C=5, penalty='l2',max_iter=2000, class_weight="balanced")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.50      0.09      0.15        22
        Good       0.83      0.98      0.90       102

    accuracy                           0.82       124
   macro avg       0.67      0.54      0.53       124
weighted avg       0.77      0.82      0.77       124

